Read DF's

In [ ]:
import numpy as np
import pandas as pd

us_trending_df = pd.read_csv("./data/stage_1_us_trending.csv")
color_text_df = pd.read_csv("./data/stage_2_thumbnail_colors_and_text.csv")
face_df = pd.read_csv("./data/stage_2_thumbnail_face.csv")
objects_df = pd.read_csv("./data/stage_2_thumbnail_objects_vectors.csv").drop('Unnamed: 0', axis=1)

Merge DF's

In [ ]:
df = pd.concat([us_trending_df, color_text_df, face_df, objects_df], axis=1)
df.head()

Calculate correlations between attributes and target class

In [ ]:
from matplotlib import pyplot as plt

target_class = 'likes'

corr_dct = dict(df.corr(method='spearman')[target_class].drop(target_class).dropna())
corr_dct = dict(sorted(corr_dct.items(), key=lambda item: item[1], reverse=True))

plt.scatter(range(len(corr_dct)), corr_dct.values())

Find attributes with small correlation

In [ ]:
for k in list(corr_dct.keys()):
    if abs(corr_dct[k]) < 0.2:
        corr_dct.pop(k)

plt.scatter(range(len(corr_dct)), corr_dct.values())

Drop attributes with small correlation from data and save to csv

In [ ]:
columns_to_drop = [ele for ele in list(df.columns) if ele not in list(corr_dct.keys())]
df.drop(columns=columns_to_drop, axis=1, inplace=True)
df.to_csv("./data/stage_3_feature_importance.csv")
df.head()